# 1. Creamos diccionario de metadatos

In [151]:
import pandas as pd
import pickle as pk
import numpy as np
from sklearn.utils import Bunch

In [152]:
#Creamos el diccionario con la estructura indicada
filepath = 'PC-GITA/PCGITA_metadata.csv'
dic_audios_inf = dict()
with open(filepath) as fp:
    cnt = 0
    for line in fp:
        line = line.split(';')[:8]
        if not line[0].startswith('AVP'):
            line[0]=line[0][3:]
        dic_audios_inf[line[0]]=dict()
        dic_audios_inf[line[0]]['UPDRS'] = 0 if line[1]=='' else int(line[1])
        dic_audios_inf[line[0]]['UPDRS_SPEECH'] = 0 if line[2]=='' else int(line[2])
        dic_audios_inf[line[0]]['HY'] = 0 if line[3]=='' else int(line[3][0])
        dic_audios_inf[line[0]]['SEX'] = 0 if line[4]=='M' else 1
        dic_audios_inf[line[0]]['AGE'] = int(line[5])
fp.close()
metadata_df = pd.DataFrame(dic_audios_inf).transpose()
metadata_df.head(10)

,AGE,HY,SEX,UPDRS,UPDRS_SPEECH
AVPEPUDEA0001,64,2,0,28,1
AVPEPUDEA0002,72,1,1,19,0
AVPEPUDEA0003,75,3,1,52,2
AVPEPUDEA0005,65,2,0,32,1
AVPEPUDEA0006,66,2,1,28,1
AVPEPUDEA0007,55,2,1,30,1
AVPEPUDEA0008,60,2,1,29,1
AVPEPUDEA0009,57,3,1,41,1
AVPEPUDEA0010,51,3,1,38,2
AVPEPUDEA0011,55,3,1,43,2


# 2. DISVOICE
## 2.1. Adición a características ya extraídas

**Anteriormente hemos extraido características y limpiado**.

**Debemos identificar en qué conjuntos de datos hemos borrado alguna instancia (limpieza de NaN) para volver a sacar esos datos.**

**En los conjuntos que no hayamos borrado ninguna instancia, podremos ahorrarnos la reextracción**

In [124]:
import cargaDatosEdSx as loader

In [125]:
re_extract = []
completos = []
for ld in dir(loader):
    if ld.startswith('load'): 
        forma = eval("loader."+str(ld)+'()[\'data\']').shape
        if forma[0]==100 or forma[0]==300:
            completos.append(ld)
        else:
            re_extract.append(ld)


In [126]:
completos

['load_art_rt',
 'load_art_w_atleta',
 'load_art_w_braso',
 'load_art_w_campana',
 'load_art_w_gato',
 'load_art_w_petaka',
 'load_fon_rt',
 'load_fon_v_A',
 'load_fon_v_E',
 'load_fon_v_I',
 'load_fon_v_O',
 'load_fon_v_U',
 'load_fon_w_campana',
 'load_prs_rt']

In [127]:
re_extract

['load_fon_w_atleta',
 'load_fon_w_braso',
 'load_fon_w_gato',
 'load_fon_w_petaka']

In [136]:
for c in completos:
    bunch = eval("loader."+str(c)+'()')
    forma = bunch['data'].shape
    if forma[0]==100:
        bunch['UPDRS'] = np.array(metadata_df['UPDRS'])
        bunch['UPDRS_SPEECH'] = np.array(metadata_df['UPDRS_SPEECH'])
        bunch['HY'] = np.array(metadata_df['HY'])
    elif forma[0]==300:
        bunch['UPDRS'] = np.array(np.repeat(metadata_df['UPDRS'],3))
        bunch['UPDRS_SPEECH'] = np.array(np.repeat(metadata_df['UPDRS_SPEECH'],3))
        bunch['HY'] = np.array(np.repeat(metadata_df['HY'],3))
    else:
        print('FALLO')
    pickle_out = open('CaracteristicasExtraidas\Regression\Orozco\\'+c[5:]+'_ccas.bunch',"wb")
    pk.dump(bunch, pickle_out)
    pickle_out.close()

## 2.2. Extraccion ccas Disvoice para 4 datasets

In [129]:
from extractorCcas import ExtractorCaracteristicas
re_extract

['load_fon_w_atleta',
 'load_fon_w_braso',
 'load_fon_w_gato',
 'load_fon_w_petaka']

Extraemos las características

In [130]:
def phonation_word_extraction(palabras):
    '''
    Llamamos a la función de extracción de características con las rutas necesarias
    '''
    ccas_palabras = dict()
    for p in palabras:
        extractor = ExtractorCaracteristicas('PC-GITA/words/'+p+'/','CaracteristicasExtraidas/Regression/Orozco/', dic_audios_inf)
        ccas_palabras[p]= extractor.extraccion_ccas_directorio('phonation', 'fon_w_'+p+'_hc.txt' , 'fon_w_'+p+'_pd.txt', ['AGE','SEX'] )
        print('Palabras analizadas: ',ccas_palabras.keys())
    return ccas_palabras

In [131]:
words=['atleta','gato','petaka','braso']
fon_words_ccas = phonation_word_extraction(words)
for k in fon_words_ccas:
    np.save('CaracteristicasExtraidas/Regression/Orozco/fon_w_'+k+'_ccas',fon_words_ccas[k])

Directorio de características ya existente, no se crea nuevo.
Palabras analizadas:  dict_keys(['atleta'])
Directorio de características ya existente, no se crea nuevo.
Palabras analizadas:  dict_keys(['atleta', 'gato'])
Directorio de características ya existente, no se crea nuevo.
Palabras analizadas:  dict_keys(['atleta', 'gato', 'petaka'])
Directorio de características ya existente, no se crea nuevo.
Palabras analizadas:  dict_keys(['atleta', 'gato', 'petaka', 'braso'])


Limpiamos los datos y Creamos objeto bunch con datos, label binaria, y labels numéricas (UPDRS, UPDRS speech e HY), teniendo en cuenta los índices de datos que hemos borado a la hora de obtener los labels.

In [132]:
ex = ExtractorCaracteristicas('daigual','CaracteristicasExtraidas/Regression/Orozco/')
dc = ex.identificadorNan('CaracteristicasExtraidas/Regression/Orozco/', True)

Directorio de características ya existente, no se crea nuevo.

--------------
 fon_w_atleta_ccas.npy
	(Audios, atrib):  (100, 32)
	Audios con NaN:  {57}
	Numero de audios con NaN:  1
	% de Nan en audios PD:  1.0

--------------
 fon_w_braso_ccas.npy
	(Audios, atrib):  (100, 32)
	Audios con NaN:  {28, 69, 15}
	Numero de audios con NaN:  3
	% de Nan en audios PD:  0.3333333333333333

--------------
 fon_w_gato_ccas.npy
	(Audios, atrib):  (100, 32)
	Audios con NaN:  {15, 18, 19, 27, 28, 34, 36, 37, 42, 45, 57, 60, 61, 67, 69, 70, 73, 80, 81, 83, 88, 91, 96, 98}
	Numero de audios con NaN:  24
	% de Nan en audios PD:  0.5833333333333334

--------------
 fon_w_petaka_ccas.npy
	(Audios, atrib):  (100, 32)
	Audios con NaN:  {57, 45, 77, 13}
	Numero de audios con NaN:  4
	% de Nan en audios PD:  0.5


In [133]:
ex.tratamiento_nan('CaracteristicasExtraidas/Regression/Orozco/', dc)
ex.identificadorNan('CaracteristicasExtraidas/Regression/Orozco/', True)

{}

In [134]:
for dtst in dc:
    data = np.load('CaracteristicasExtraidas/Regression/Orozco/'+dtst)
    
    attributes = data[:,:-1]
    label_binary = data[:,-1]
    
    labels_limpias = metadata_df.drop(metadata_df.index[ list(dc[dtst][0])])
    
    updrs = np.array(labels_limpias['UPDRS'])
    updrs_speech = np.array(labels_limpias['UPDRS_SPEECH'])
    hy =  np.array(labels_limpias['HY'])
    
    bunch = Bunch(data = attributes, 
                  target = label_binary, 
                  UPDRS = updrs, 
                  UPDRS_SPEECH = updrs_speech, 
                  HY = hy )
    
    pickle_out = open('CaracteristicasExtraidas\Regression\Orozco\\'+dtst[:-4]+'.bunch',"wb")
    pk.dump(bunch, pickle_out)
    pickle_out.close()
    
    print(dtst)
    print(attributes.shape)
    print(label_binary.shape)
    print(updrs.shape)
    print('--------------')

fon_w_atleta_ccas.npy
(99, 31)
(99,)
(99,)
--------------
fon_w_braso_ccas.npy
(97, 31)
(97,)
(97,)
--------------
fon_w_gato_ccas.npy
(76, 31)
(76,)
(76,)
--------------
fon_w_petaka_ccas.npy
(96, 31)
(96,)
(96,)
--------------


Borramos temporales

In [137]:
import os
for f in os.listdir('CaracteristicasExtraidas\Regression\Orozco\\'):
    if f.endswith('npy') or f.endswith('txt'):
        os.remove('CaracteristicasExtraidas\Regression\Orozco\\'+f)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'CaracteristicasExtraidas\\Regression\\Orozco\\fon_w_petaka_ccas.npy'

In [150]:
bn = pk.load(open(r'CaracteristicasExtraidas\Regression\Orozco\fon_w_petaka_ccas.bunch','rb'))
print(type(bn))
for k in bn.keys():
    print(bn[k].shape)

<class 'sklearn.utils.Bunch'>
(96, 31)
(96,)
(96,)
(96,)
(96,)


# 3. VGGish

In [14]:
import cargaDatosVggishEmbeddings as loaderVggish

In [142]:
vggish_re_extract = []
vggish_completos = []
for ld in dir(loaderVggish):
    if ld.startswith('load'): 
        forma = eval("loaderVggish."+str(ld)+'()[\'data\']').shape
        print(ld, forma)
        if forma[0]==100 or forma[0]==300:
            vggish_completos.append(ld)
        else:
            vggish_re_extract.append(ld)


load_vggish_embed_rt (100, 258)
load_vggish_embed_v_A (282, 256)
load_vggish_embed_v_E (284, 256)
load_vggish_embed_v_I (281, 256)
load_vggish_embed_v_O (262, 256)
load_vggish_embed_v_U (254, 256)


In [140]:
vggish_completos

['load_vggish_embed_rt']

In [141]:
vggish_re_extract

['load_vggish_embed_v_A',
 'load_vggish_embed_v_E',
 'load_vggish_embed_v_I',
 'load_vggish_embed_v_O',
 'load_vggish_embed_v_U']